In [93]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm
from midi_player import MIDIPlayer
import os

In [95]:
bpm2tempo(120)

500000

In [41]:
data_path = '_adl-piano-midi/midi/adl-piano-midi'
total_midi_count = 0
for path in os.listdir(data_path):
    midi_count = 0
    for _, _, files in os.walk(os.path.join(data_path, path)):
        for file in files:
            if file.endswith('.mid'): midi_count += 1
    print(path, midi_count)
    total_midi_count += midi_count

print('Total:', total_midi_count)

Pop 750
.DS_Store 0
Blues 129
Latin 253
Ambient 185
World 503
Reggae 26
Electronic 161
Classical 1398
Rock 3137
Religious 55
Folk 90
Unknown 2171
Soul 351
Children 24
Soundtracks 947
Country 246
Rap 158
Jazz 492
Total: 11076


In [42]:
# TODO: Create the midi parser.
# Following https://arxiv.org/pdf/1808.03715 - 5.0.1

In [43]:
midi_path = './_adl-piano-midi/midi/adl-piano-midi/Classical/Classical Flute/James Galway/Hero.mid'
# midi_path = './_adl-piano-midi/midi/adl-piano-midi/Folk/Alpine Yodeling/Jacob Sisters/La-La-La.mid'

In [44]:
mid = mido.MidiFile(midi_path)
for track in mid.tracks:
    for msg in track:
        if msg.type in ['note_on']:
            print('\t', msg)
        else:
            print(msg)

MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='E', time=0)
MetaMessage('end_of_track', time=1)
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=110 time=0
control_change channel=0 control=64 value=127 time=0
	 note_on channel=0 note=66 velocity=100 time=0
	 note_on channel=0 note=68 velocity=100 time=0
	 note_on channel=0 note=71 velocity=100 time=0
	 note_on channel=0 note=64 velocity=100 time=60
	 note_on channel=0 note=71 velocity=0 time=90
	 note_on channel=0 note=71 velocity=100 time=0
	 note_on channel=0 note=71 velocity=0 time=30
	 note_on channel=0 note=76 velocity=100 time=0
	 note_on channel=0 note=75 velocity=100 time=30
	 note_on channel=0 note=76 velocity=0 time=2
	 note_on channel=0 note=75 velocity=0 time=28
	 note_on channel=0 note=78 velocity=100 time=5
	 note_on channel=0 note=76 velo

In [46]:
def get_instructions():
    pass

In [146]:
def midi_to_tensor(midi_path):

    mid = mido.MidiFile(midi_path)
    instruction_library = {
        'time_shift': lambda x: ('time_shift', x),
        'note_on': lambda x: ('note_on', x),
        'note_off': lambda x: ('note_off', x),
        'velocity': lambda x: ('velocity', x)
    }

    midi_instructions = []
    for track in mid.tracks:

        on_notes = []
        last_velocity = -1
        sustain = False

        for msg in track:

            # Note On
            if (msg.type == 'note_on') and (msg.velocity != 0):
                on_notes.append(msg.note)
                if msg.time > 0:
                    midi_instructions.append(instruction_library['time_shift'](msg.time))

                velocity_bucket = msg.velocity // 4
                if last_velocity != velocity_bucket:
                    midi_instructions.append(instruction_library['velocity'](velocity_bucket))
                    last_velocity = velocity_bucket
                    
                midi_instructions.append(instruction_library['note_on'](msg.note))

            # Note Off
            elif (msg.type == 'note_off') or ((msg.type == 'note_on') and (msg.velocity == 0)):
                if msg.time > 0:
                    midi_instructions.append(instruction_library['time_shift'](msg.time))
                if sustain:
                    continue
                midi_instructions.append(instruction_library['note_off'](msg.note))
                if msg.note in on_notes:
                    on_notes.remove(msg.note)

            # Sustain Pedal
            elif (msg.type == 'control_change') and (msg.control == 64):
                if msg.value < 64: # Sustain pedal off
                    sustain = False
                    if msg.time > 0:
                        midi_instructions.append(instruction_library['time_shift'](msg.time))
                    # Turn all notes off
                    for on_note in on_notes:
                        midi_instructions.append(instruction_library['note_off'](on_note))
                    on_notes = []
                else:
                    sustain = True
                    if msg.time > 0:
                        midi_instructions.append(instruction_library['time_shift'](msg.time))

            else:
                continue

    return midi_instructions

In [147]:
test_midi_path = 'test.mid'
def tensor_to_midi(midi_instructions, save_path=None):
    midi_out = MidiFile()
    # midi_out.tracks.append(mid.tracks[0])

    track = MidiTrack()
    track.append(MetaMessage('key_signature', key='E'))
    track.append(MetaMessage('set_tempo', tempo=bpm2tempo(15)))
    track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

    time_shift = 0
    d_velocity = 0
    for instruction in midi_instructions:
        if instruction[0] == 'time_shift':
            time_shift += instruction[1]
        elif instruction[0] == 'velocity':
            d_velocity = instruction[1]
        else:
            track.append(Message(instruction[0], note=instruction[1], velocity=d_velocity*4, time=time_shift))
            time_shift = 0

    track.append(MetaMessage('end_of_track', time=1))
    midi_out.tracks.append(track)

    if save_path:
        midi_out.save(save_path)

    return midi_out

In [148]:
midi_instructions = midi_to_tensor(midi_path)
midi_out = tensor_to_midi(midi_instructions, test_midi_path)

In [149]:
MIDIPlayer(test_midi_path, 400)

In [45]:
MIDIPlayer(midi_path, 400)

In [55]:
for track in midi_out.tracks:
    print('\t---')

    on_notes = []
    last_velocity = None

    for msg in track:
        print(msg)

	---
MetaMessage('key_signature', key='E', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
program_change channel=0 program=12 time=10
note_on channel=0 note=66 velocity=0 time=0
note_on channel=0 note=68 velocity=0 time=0
note_on channel=0 note=71 velocity=0 time=0
note_on channel=0 note=64 velocity=0 time=60
note_off channel=0 note=71 velocity=0 time=90
note_on channel=0 note=71 velocity=0 time=0
note_off channel=0 note=71 velocity=0 time=30
note_on channel=0 note=76 velocity=0 time=0
note_on channel=0 note=75 velocity=0 time=30
note_off channel=0 note=76 velocity=0 time=2
note_off channel=0 note=75 velocity=0 time=28
note_on channel=0 note=78 velocity=0 time=5
note_on channel=0 note=76 velocity=0 time=55
note_off channel=0 note=78 velocity=0 time=5
note_off channel=0 note=76 velocity=0 time=25
note_on channel=0 note=76 velocity=0 time=0
note_on channel=0 note

In [23]:
for track in mid.tracks:
    print('\t---')

    on_notes = []
    last_velocity = None

    print(track[:3])
    continue

    for msg in track:
        print(msg)

	---
MidiTrack([
  MetaMessage('set_tempo', tempo=1000000, time=0),
  MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('key_signature', key='E', time=0)])
	---
MidiTrack([
  Message('program_change', channel=0, program=0, time=0),
  Message('control_change', channel=0, control=7, value=110, time=0),
  Message('control_change', channel=0, control=64, value=127, time=0)])


In [17]:
arr = [2, 3, 4]
arr.remove(3)
arr

[2, 4]

In [38]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(MetaMessage('key_signature', key='Dm'))
track.append(MetaMessage('set_tempo', tempo=1000000))
track.append(MetaMessage('time_signature', numerator=6, denominator=8))

track.append(Message('program_change', program=12, time=10))
track.append(Message('note_on', channel=2, note=60, velocity=64, time=1))
track.append(Message('note_off', channel=2, note=60, velocity=100, time=2))

track.append(MetaMessage('end_of_track'))

mid.save('new_song.mid')

In [39]:
MIDIPlayer('new_song.mid', 400)